In [68]:
%store -r data
%store -r le

In [69]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly 
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [70]:
init_notebook_mode(connected=True)

In [71]:
data.head(2)

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,duration,state_num
ID,,,,,,,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,160,43,23,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,False,0,22,0.0,0.0,1533.95,58 days 11:47:32,0
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,145,20,51,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,False,15,50,100.0,2421.0,30000.00,59 days 19:16:03,0


In [46]:
data["duration"] = pd.to_timedelta(data["deadline"]  - data['launched'])

In [47]:
success_proj = data[data["state"] == True]
suc_cat = pd.DataFrame(success_proj.main_category.value_counts())
suc_cat.columns = ["suc_main_cat"]
suc_cat["total"] = data.main_category.value_counts()
suc_cat["rate"] = suc_cat.suc_main_cat / suc_cat.total * 100
suc_cat.index = le.inverse_transform(suc_cat.index)
suc_cat.sort_values(by="rate",inplace=True)

In [26]:
gdata = [go.Bar(x = suc_cat.index, y = suc_cat.suc_main_cat, name = "Success Number"),
        go.Bar(x = suc_cat.index, y = suc_cat.total,name = "Total Number"),
        go.Scatter(x = suc_cat.index, y = suc_cat.rate,yaxis='y2', name = "Success Rate")
       ]

In [27]:
iplot(go.Figure(data = gdata, 
        layout = go.Layout(
            title='Main Catagory Successful Rate',
        yaxis=dict(
            title='Number of Projects'
        ),
        yaxis2=dict(
            title='Successful Rate',
            tickfont=dict(
                color='rgb(148, 103, 189)'
            ),
            overlaying='y',
            side='right'
        )
        )
     )
     )

> It is suprising that low successful rate of technology category. I guess X

In [28]:
success_proj = data[data["state"] == True]
suc_sub_cat = pd.DataFrame(success_proj.category.value_counts())
suc_sub_cat.columns = ["suc_cat"]
suc_sub_cat["total"] = data.category.value_counts()
suc_sub_cat["rate"] = suc_sub_cat.suc_cat / suc_sub_cat.total * 100
suc_sub_cat.index = le.inverse_transform(suc_sub_cat.index)
suc_sub_cat.sort_values(by="rate",inplace=True)

gdata = [go.Bar(x = suc_sub_cat.index, y = suc_sub_cat.suc_cat, name = "Success Number"),
        go.Bar(x = suc_sub_cat.index, y = suc_sub_cat.total,name = "Total Number"),
        go.Scatter(x = suc_sub_cat.index, y = suc_sub_cat.rate,yaxis='y2', name = "Success Rate")
       ]

iplot(go.Figure(data = gdata, 
        layout = go.Layout(
            title='Sub Catagory Successful Rate',
        yaxis=dict(
            title='Number of Projects'
        ),
        yaxis2=dict(
            title='Successful Rate',
            tickfont=dict(
                color='rgb(148, 103, 189)'
            ),
            overlaying='y',
            side='right'
        )
        )
     )
     )

In [347]:
data.head(2)

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,duration
ID,,,,,,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,160,43,23,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,False,0,22,0.0,0.0,1533.95,58 days 11:47:32
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,145,20,51,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,False,15,50,100.0,2421.0,30000.00,59 days 19:16:03


# How the goal affect the successful rate

In [160]:
gsource = []
gtarget =  []
gvalue =  []
glabel =  ['True', "False"] +[str(x) for x in list(set(sorted(pd.cut(data.goal,15000,right=False))))]

In [161]:
temp = sorted(list(set(zip([str(x) for x in list(pd.cut(data.goal,15000,right=False))],
                           data.state.astype(str)))),key=lambda x: x[1])
data_temp =list(zip([str(x) for x in list(pd.cut(data.goal,15000,right=False))],data.state.astype(str)))

In [162]:
value = [data_temp.count(x) for x in temp]
gsource += [glabel.index(x[0]) for x in temp]
gtarget += [glabel.index(x[1]) for x in temp]
gvalue += value

In [164]:
data_trace = dict(
    type='sankey',
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(
        width = 0.5
      ),
        label = glabel
    ),
    link = dict(
      source =  gsource,
      target =  gtarget,
      value =  gvalue
  )
)
layout =  go.Layout(
    title = "Kickstarter Campaign Flow (Goal -> Successful?)",
    font = dict(
      size = 10
    )
)
fig = go.Figure(data=[data_trace],layout = layout)
iplot(fig, validate=True)

> The value is the number of campaigns which are in that group. We grouped the goals into 15000 groups and we found out some of the goals which set by the campaign were never successful. 

# How duration of the campaign affects the successful rate

In [169]:
temp = sorted(list(set(zip([str(x) for x in list(pd.cut(data.duration,5000,right=False))],
                           data.state.astype(str)))),key=lambda x: x[1])

In [171]:
gsource = []
gtarget =  []
gvalue =  []
glabel =  ['True', "False"] +[str(x) for x in list(set(sorted(pd.cut(data.duration,5000,right=False))))]
temp = sorted(list(set(zip([str(x) for x in list(pd.cut(data.duration,5000,right=False))],
                           data.state.astype(str)))),key=lambda x: x[1])
data_temp =list(zip([str(x) for x in list(pd.cut(data.duration,5000,right=False))],data.state.astype(str)))

In [172]:
value = [data_temp.count(x) for x in temp]
gsource += [glabel.index(x[0]) for x in temp]
gtarget += [glabel.index(x[1]) for x in temp]
gvalue += value

In [173]:
data_trace = dict(
    type='sankey',
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(
        width = 0.5
      ),
        label = glabel
    ),
    link = dict(
      source =  gsource,
      target =  gtarget,
      value =  gvalue
  )
)
layout =  go.Layout(
    title = "Kickstarter Campaign Flow (Goal -> Successful?)",
    font = dict(
      size = 10
    )
)
fig = go.Figure(data=[data_trace],layout = layout)
iplot(fig, validate=True)

> As what we see in the previous graph. Some of the duation group of campaign never succeed. In the prediction part, we may dig into this and make the prediction better. 

In [128]:
#sub -> main catagory -> country -> duration -> goal   -> state

In [348]:
gsource = []
gtarget =  []
gvalue =  []
glabel =  []

In [349]:
data["duation_cat"] = pd.cut(data.duration, bins = 1000).astype(str)
data["goal_cat"] = pd.cut(data.goal, bins = 20).astype(str)

In [371]:
data.state =data.state.astype(str)

In [373]:
subcat = list(set(le.inverse_transform(data.category)))
cat = list(set(le.inverse_transform(data.main_category)))
for i in range(len(subcat)):
    if subcat[i] in cat:
        subcat[i] = subcat[i]+"_sub"
cou = list(set(le.inverse_transform(data.country)))
dua = list(set(data.duation_cat))
goal = list(set(data.goal_cat))
total = subcat + cat+cou+dua+goal+["True", "False"]

In [387]:

# Main -> COuntry
temp = sorted(list(set(zip(data.main_category,data.country))),key=lambda x: x[1])
value = [list(zip(data.main_category,data.country)).count(x) for x in temp]
gsource += [total.index(le.inverse_transform([x[0]])[0]) for x in temp]
gtarget += [total.index(le.inverse_transform([x[1]])[0]) for x in temp]
gvalue += value

#country -> duation
temp = sorted(list(set(zip(data.country,data.duation_cat))),key=lambda x: x[1])
value = [list(zip(data.country,data.duation_cat)).count(x) for x in temp]
gsource += [total.index(le.inverse_transform([x[0]])[0]) for x in temp]
gtarget += [total.index(x[1]) for x in temp]
gvalue += value

#duation ->goal
temp = sorted(list(set(zip(data.duation_cat,data.goal_cat))),key=lambda x: x[1])
value = [list(zip(data.duation_cat,data.goal_cat)).count(x) for x in temp]
gsource += [total.index(x[0]) for x in temp]
gtarget += [total.index(x[1]) for x in temp]
gvalue += value

#goal -> state
temp = sorted(list(set(zip(data.goal_cat,data.state))),key=lambda x: x[1])
value = [list(zip(data.goal_cat,data.state)).count(x) for x in temp]
gsource += [total.index(x[0]) for x in temp]
gtarget += [total.index(x[1]) for x in temp]
gvalue += value


In [389]:
data_trace = dict(
    type='sankey',
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(
        width = 0.5
      ),
        label = total
    ),
    link = dict(
      source =  gsource,
      target =  gtarget,
      value =  gvalue
  )
)

layout =  go.Layout(
    title = "Kickstarter Campaign Flow (Sub Category -> Main Cat -> Country -> Duration -> Goal -> Successful?)",
    font = dict(
      size = 10
    )
)

fig = go.Figure(data=[data_trace],layout = layout)
iplot(fig, validate=True)